<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 52px">

# Final Project, Part 3: Technical Notebook
## Bonus Content: TripAdvisor Data Scraping

#### <br>Student: Christopher Doughty
#### Date: Oct. 9, 2021

---

### Introduction

Please do not attempt to run all the code in this notebook - it will break. All that remains here are random scraps and snatches of various approaches I tried to use to scrape the data I wanted off of TripAdvisor until I arrived at a solution that worked.

As part of my project, I realized I needed to scrape individual reviews for each ride in order to supplement the ride information if I was going to make a predictive model for ride rating/popularity/response among park-goers.

TripAdvisor is not an easy site to scrape - it's all in HTML and there is a lot of "noise" per page with ads and featured reviews for other rides (https://www.tripadvisor.com/). The information I wanted most - star ratings - are stored as svg images on TripAdvisor (https://www.w3schools.com/html/html5_svg.asp),rather than strings or numbers, making it very hard to collect that info.

Another challenge I encountered was that even if I figured out how to successfully scrape one page, the HTML classes differed for each page, so the code I used on one page would not work for another. And even if I could apply a standard code to all the pages I wanted, I realized that if I were to try to do that, my IP address would likely get blacklisted because I would need to send get requests for something like 1,000 pages. Even with a sleep function to stagger that a bit, it would take a long time to run in the notebook, espeically with sleep intervals.

The Apify API I found gave me a way to consistently scrape data across multiple pages, but it was incomplete, not able to bring in all reviews or all attractions, and what it did bring in was in a horrible csv file that made clusters of data points into features, creating a huge number of features that would have been a real headache to disentangle.

Eventually, I found a web tool that allowed me to scrape the data through an almost GUI-like interface, though even with this, the best I could do was scraping the HTML code for the rating svg images and then cleaning that feature so that only the rating number remained.

### Behold - A Graveyard of Failure

An attempt at using an Apify API I found on GitHub that *almost* worked, but not quite.

I tried this method both within Jupyter and on Apify. Using it on the Apify site seemed to work better.

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
from random import randint
from time import sleep

In [ ]:
from apify_client import ApifyClient

# Initialize the ApifyClient with your API token
client = ApifyClient("<apify_api_R79VT2EoA2dY5vreyEz4hEdHcjcJys2kBI1L>")

# Prepare the actor input
run_input = {
    "locationFullName": "Prague",
    "maxReviews": 0,
    "language": "en",
    "currency": "USD",
    "proxyConfiguration": { "useApifyProxy": True },
}

# Run the actor and wait for it to finish
run = client.actor("maxcopell/tripadvisor").call(run_input=run_input)

# Fetch and print actor results from the run's dataset (if there are any)
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    print(item)

### A failed for loop attempt that would loop through every page I wanted to scrape.

In [ ]:
URL = [
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563101-Reviews-Astro_Orbiter-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563101-Reviews-or10-Astro_Orbiter-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563101-Reviews-or20-Astro_Orbiter-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563101-Reviews-or30-Astro_Orbiter-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563101-Reviews-or40-Astro_Orbiter-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or10-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or20-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or30-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or40-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or50-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or60-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or70-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or80-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or90-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or100-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or110-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or120-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or130-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or140-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or150-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or160-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or170-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or180-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or190-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or200-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or210-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or220-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or230-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or240-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or250-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or260-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or270-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or280-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or290-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or300-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or310-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or320-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or330-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or340-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or350-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or360-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or370-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or380-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or390-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or400-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or410-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or420-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or430-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or440-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or450-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or460-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or470-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or480-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or490-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or500-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or510-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or520-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or530-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or540-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or550-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or560-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or570-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or580-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or590-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or600-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or610-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or620-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or630-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or640-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or650-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or660-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or670-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or680-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or690-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or700-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or710-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or720-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or730-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or740-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or750-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or760-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or770-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or780-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or790-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or800-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or810-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or820-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or830-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or840-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or850-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or860-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or870-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or880-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or890-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or900-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or910-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or920-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or930-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or940-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or950-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or960-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or970-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or980-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or990-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1000-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1010-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1020-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1030-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1040-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1050-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1060-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1070-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1080-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1090-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1100-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1110-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1120-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1130-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1140-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1150-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1160-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1170-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1180-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1190-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1200-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1210-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1220-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1230-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1240-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1250-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1260-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1270-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1280-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1290-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1300-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14145257-Reviews-or1310-Avatar_Flight_of_Passage-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or10-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or20-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or30-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or40-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or50-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or60-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or70-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or80-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or90-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or100-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or110-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or120-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or130-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or140-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or150-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or160-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or170-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or180-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or190-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or200-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or210-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or220-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or230-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or240-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or250-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or260-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563103-Reviews-or270-Big_Thunder_Mountain_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or10-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or20-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or30-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or40-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or50-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or60-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or70-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or80-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or90-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or100-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or110-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or120-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or130-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or140-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or150-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or160-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or170-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or180-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or190-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or200-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or210-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563105-Reviews-or220-Buzz_Lightyear_s_Space_Ranger_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-or10-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-or20-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-or30-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-or40-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-or50-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-or60-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-or70-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-or80-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-or90-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-or100-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-or110-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-or120-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-or130-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-or140-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562272-Reviews-or150-Dinosaur-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563118-Reviews-Dumbo_the_Flying_Elephant-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563118-Reviews-or10-Dumbo_the_Flying_Elephant-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563118-Reviews-or20-Dumbo_the_Flying_Elephant-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563118-Reviews-or30-Dumbo_the_Flying_Elephant-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563118-Reviews-or40-Dumbo_the_Flying_Elephant-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563118-Reviews-or50-Dumbo_the_Flying_Elephant-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563118-Reviews-or60-Dumbo_the_Flying_Elephant-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563118-Reviews-or70-Dumbo_the_Flying_Elephant-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563118-Reviews-or80-Dumbo_the_Flying_Elephant-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563118-Reviews-or90-Dumbo_the_Flying_Elephant-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or10-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or20-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or30-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or40-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or50-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or60-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or70-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or80-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or90-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or100-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or110-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or120-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or130-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or140-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or150-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or160-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or170-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or180-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or190-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or200-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or210-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or220-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or230-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or240-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or250-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or260-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or270-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or280-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or290-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or300-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or310-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or320-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or330-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or340-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563259-Reviews-or350-Expedition_Everest_Legend_of_the_Forbidden_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d13535055-Reviews-Frozen_Ever_After-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d13535055-Reviews-or10-Frozen_Ever_After-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d13535055-Reviews-or20-Frozen_Ever_After-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d13535055-Reviews-or30-Frozen_Ever_After-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d13535055-Reviews-or40-Frozen_Ever_After-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d13535055-Reviews-or50-Frozen_Ever_After-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d13535055-Reviews-or60-Frozen_Ever_After-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d13535055-Reviews-or70-Frozen_Ever_After-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d13535055-Reviews-or80-Frozen_Ever_After-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d13535055-Reviews-or90-Frozen_Ever_After-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d13535055-Reviews-or100-Frozen_Ever_After-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d13535055-Reviews-or110-Frozen_Ever_After-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or5-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or10-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or15-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or20-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or25-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or30-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or35-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or40-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or45-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or50-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or55-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or60-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or65-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or70-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or75-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or80-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or85-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or90-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or95-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or100-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563192-Reviews-or105-Gran_Fiesta_Tour_Starring_The_Three_Caballeros-Orlando_Florida.html#REVIEWS',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or10-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or20-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or30-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or40-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or50-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or60-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or70-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or80-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or90-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or100-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or110-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or120-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or130-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or140-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or150-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or160-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or170-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or180-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or190-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or200-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or210-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or220-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or230-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or240-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or250-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or260-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or270-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or280-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or290-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or300-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or310-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or320-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562260-Reviews-or330-Haunted_Mansion-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or10-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or20-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or30-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or40-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or50-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or60-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or70-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or80-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or90-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or100-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or110-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or120-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or130-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or140-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or150-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or160-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or170-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or180-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or190-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or200-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or210-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or220-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or230-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563140-Reviews-or240-It_s_a_Small_World-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563206-Reviews-Journey_Into_Imagination_with_Figment-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563206-Reviews-or10-Journey_Into_Imagination_with_Figment-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563206-Reviews-or20-Journey_Into_Imagination_with_Figment-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563206-Reviews-or30-Journey_Into_Imagination_with_Figment-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563206-Reviews-or40-Journey_Into_Imagination_with_Figment-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563206-Reviews-or50-Journey_Into_Imagination_with_Figment-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563206-Reviews-or60-Journey_Into_Imagination_with_Figment-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563206-Reviews-or70-Journey_Into_Imagination_with_Figment-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563206-Reviews-or80-Journey_Into_Imagination_with_Figment-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563206-Reviews-or90-Journey_Into_Imagination_with_Figment-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563206-Reviews-or100-Journey_Into_Imagination_with_Figment-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563206-Reviews-or110-Journey_Into_Imagination_with_Figment-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563206-Reviews-or120-Journey_Into_Imagination_with_Figment-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or10-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or20-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or30-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or40-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or50-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or60-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or70-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or80-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or90-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or100-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or110-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or120-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or130-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or140-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or150-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or160-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or170-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or180-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or190-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or200-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or210-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563123-Reviews-or220-Jungle_Cruise-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563270-Reviews-Kali_River_Rapids-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563270-Reviews-or10-Kali_River_Rapids-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563270-Reviews-or20-Kali_River_Rapids-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563270-Reviews-or30-Kali_River_Rapids-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563270-Reviews-or40-Kali_River_Rapids-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563270-Reviews-or50-Kali_River_Rapids-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563270-Reviews-or60-Kali_River_Rapids-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563270-Reviews-or70-Kali_River_Rapids-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563270-Reviews-or80-Kali_River_Rapids-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563270-Reviews-or90-Kali_River_Rapids-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563270-Reviews-or100-Kali_River_Rapids-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563270-Reviews-or110-Kali_River_Rapids-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563270-Reviews-or120-Kali_River_Rapids-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or10-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or20-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or30-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or40-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or50-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or60-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or70-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or80-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or90-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or100-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or110-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or120-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or130-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or140-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or150-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or160-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or170-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or180-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or190-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or200-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or210-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or220-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or230-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or240-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or250-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or260-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or270-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or280-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or290-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563269-Reviews-or300-Kilimanjaro_Safaris-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or10-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or20-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or30-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or40-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or50-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or60-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or70-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or80-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or90-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or100-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or110-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or120-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or130-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or140-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or150-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or160-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or170-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or180-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563210-Reviews-or190-Living_with_the_Land-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563124-Reviews-Mad_Tea_Party-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563124-Reviews-or10-Mad_Tea_Party-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563124-Reviews-or20-Mad_Tea_Party-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563124-Reviews-or30-Mad_Tea_Party-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563124-Reviews-or40-Mad_Tea_Party-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563124-Reviews-or50-Mad_Tea_Party-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563124-Reviews-or60-Mad_Tea_Party-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563127-Reviews-Main_Street_Vehicles-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563127-Reviews-or10-Main_Street_Vehicles-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or10-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or20-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or30-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or40-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or50-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or60-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or70-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or80-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or90-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or100-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or110-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or120-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or130-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or140-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or150-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or160-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or170-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or180-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or190-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or200-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or210-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or220-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563212-Reviews-or230-Mission_SPACE-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14142516-Reviews-Na_vi_River_Journey-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14142516-Reviews-or10-Na_vi_River_Journey-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14142516-Reviews-or20-Na_vi_River_Journey-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14142516-Reviews-or30-Na_vi_River_Journey-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14142516-Reviews-or40-Na_vi_River_Journey-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14142516-Reviews-or50-Na_vi_River_Journey-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14142516-Reviews-or60-Na_vi_River_Journey-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14142516-Reviews-or70-Na_vi_River_Journey-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14142516-Reviews-or80-Na_vi_River_Journey-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14142516-Reviews-or90-Na_vi_River_Journey-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14142516-Reviews-or100-Na_vi_River_Journey-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14142516-Reviews-or110-Na_vi_River_Journey-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14142516-Reviews-or120-Na_vi_River_Journey-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14142516-Reviews-or130-Na_vi_River_Journey-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d14142516-Reviews-or140-Na_vi_River_Journey-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or10-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or20-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or30-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or40-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or50-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or60-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or70-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or80-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or90-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or100-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or110-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or120-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or130-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or140-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or150-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or160-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or170-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or180-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or190-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or200-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563131-Reviews-or210-Peter_Pan_s_Flight-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or10-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or20-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or30-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or40-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or50-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or60-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or70-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or80-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or90-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or100-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or110-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or120-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or130-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or140-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or150-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or160-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or170-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or180-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or190-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or200-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or210-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or220-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or230-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or240-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or250-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or260-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or270-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or280-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or290-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or300-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563134-Reviews-or310-Pirates_of_the_Caribbean-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563273-Reviews-Primeval_Whirl-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563273-Reviews-or10-Primeval_Whirl-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563273-Reviews-or20-Primeval_Whirl-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563273-Reviews-or30-Primeval_Whirl-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563273-Reviews-or40-Primeval_Whirl-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563273-Reviews-or50-Primeval_Whirl-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563273-Reviews-or60-Primeval_Whirl-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563273-Reviews-or70-Primeval_Whirl-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563137-Reviews-Prince_Charming_Regal_Carrousel-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563137-Reviews-or10-Prince_Charming_Regal_Carrousel-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563137-Reviews-or20-Prince_Charming_Regal_Carrousel-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563137-Reviews-or30-Prince_Charming_Regal_Carrousel-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563137-Reviews-or40-Prince_Charming_Regal_Carrousel-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or10-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or20-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or30-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or40-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or50-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or60-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or70-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or80-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or90-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or100-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or110-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or120-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or130-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or140-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or150-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or160-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or170-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or180-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or190-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or200-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or210-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or220-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or230-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or240-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or250-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or260-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563302-Reviews-or270-Rock_n_Roller_Coaster_Starring_Aerosmith-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or10-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or20-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or30-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or40-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or50-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or60-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or70-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or80-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or90-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or100-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or110-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or120-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or130-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or140-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or150-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or160-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or170-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or180-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or190-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or200-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or210-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or220-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or230-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or240-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or250-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or260-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or270-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or280-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or290-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or300-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or310-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or320-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or330-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or340-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or350-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or360-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or370-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or380-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563138-Reviews-or390-Seven_Dwarfs_Mine_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d17523143-Reviews-Slinky_Dog_Dash-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d17523143-Reviews-or10-Slinky_Dog_Dash-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d17523143-Reviews-or20-Slinky_Dog_Dash-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d17523143-Reviews-or30-Slinky_Dog_Dash-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or10-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or20-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or30-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or40-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or50-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or60-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or70-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or80-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or90-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or100-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or110-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or120-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or130-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or140-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or150-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or160-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or170-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or180-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or190-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or200-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or210-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or220-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or230-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or240-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or250-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or260-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or270-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or280-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or290-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or300-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or310-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or320-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or330-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or340-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or350-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or360-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or370-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or380-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or390-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or400-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or410-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or420-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or430-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or440-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or450-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or460-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or470-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or480-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or490-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or500-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or510-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or520-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or530-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or540-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or550-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563234-Reviews-or560-Soarin-Orlando_Florida.html'
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or10-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or20-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or30-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or40-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or50-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or60-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or70-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or80-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or90-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or100-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or110-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or120-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or130-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or140-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or150-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or160-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or170-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or180-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or190-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or200-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or210-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or220-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or230-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or240-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or250-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or260-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or270-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or280-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or290-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or300-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or310-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563169-Reviews-or320-Space_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or10-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or20-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or30-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or40-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or50-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or60-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or70-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or80-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or90-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or100-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or110-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or120-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or130-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or140-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or150-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or160-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or170-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or180-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or190-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or200-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or210-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or220-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or230-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or240-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563236-Reviews-or250-Spaceship_Earth-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or10-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or20-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or30-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or40-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or50-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or60-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or70-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or80-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or90-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or100-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or110-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or120-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or130-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or140-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or150-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or160-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or170-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or180-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or190-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or200-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or210-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or220-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or230-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or240-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or250-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or260-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or270-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or280-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563157-Reviews-or290-Splash_Mountain-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or10-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or20-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or30-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or40-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or50-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or60-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or70-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or80-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or90-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or100-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or110-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or120-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or130-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or140-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or150-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or160-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or170-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or180-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or190-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or200-Star_Tours_The_Adventures_Continue-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or10-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or20-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or30-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or40-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or50-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or60-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or70-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or80-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or90-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or100-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or110-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or120-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or130-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or140-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or150-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or160-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or170-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or180-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or190-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or200-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or210-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or220-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or230-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or240-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or250-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or260-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or270-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563231-Reviews-or280-Test_Track_Presented_by_Chevrolet-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563116-Reviews-The_Barnstormer-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563116-Reviews-or10-The_Barnstormer-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563116-Reviews-or20-The_Barnstormer-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563116-Reviews-or30-The_Barnstormer-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563116-Reviews-or40-The_Barnstormer-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563116-Reviews-or50-The_Barnstormer-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563116-Reviews-or60-The_Barnstormer-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563116-Reviews-or70-The_Barnstormer-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562267-Reviews-The_Magic_Carpets_of_Aladdin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562267-Reviews-or10-The_Magic_Carpets_of_Aladdin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562267-Reviews-or20-The_Magic_Carpets_of_Aladdin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562267-Reviews-or30-The_Magic_Carpets_of_Aladdin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562267-Reviews-or40-The_Magic_Carpets_of_Aladdin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562267-Reviews-or50-The_Magic_Carpets_of_Aladdin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8562267-Reviews-or60-The_Magic_Carpets_of_Aladdin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563128-Reviews-The_Many_Adventures_of_Winnie_the_Pooh-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563128-Reviews-or10-The_Many_Adventures_of_Winnie_the_Pooh-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563128-Reviews-or20-The_Many_Adventures_of_Winnie_the_Pooh-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563128-Reviews-or30-The_Many_Adventures_of_Winnie_the_Pooh-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563128-Reviews-or40-The_Many_Adventures_of_Winnie_the_Pooh-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563128-Reviews-or50-The_Many_Adventures_of_Winnie_the_Pooh-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563128-Reviews-or60-The_Many_Adventures_of_Winnie_the_Pooh-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563128-Reviews-or70-The_Many_Adventures_of_Winnie_the_Pooh-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563128-Reviews-or80-The_Many_Adventures_of_Winnie_the_Pooh-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563128-Reviews-or90-The_Many_Adventures_of_Winnie_the_Pooh-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563128-Reviews-or100-The_Many_Adventures_of_Winnie_the_Pooh-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563128-Reviews-or110-The_Many_Adventures_of_Winnie_the_Pooh-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or10-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or20-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or30-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or40-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or50-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or60-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or70-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or80-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or90-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or100-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or110-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or120-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or130-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or140-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or150-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or160-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or170-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or180-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or190-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or200-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or210-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or220-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or230-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or240-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or250-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or260-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or270-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or280-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or290-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or300-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or310-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or320-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or330-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or340-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or350-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or360-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or370-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563303-Reviews-or380-The_Twilight_Zone_Tower_of_Terror-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563164-Reviews-Tomorrowland_Speedway-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563164-Reviews-or10-Tomorrowland_Speedway-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563164-Reviews-or20-Tomorrowland_Speedway-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563164-Reviews-or30-Tomorrowland_Speedway-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563164-Reviews-or40-Tomorrowland_Speedway-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563164-Reviews-or50-Tomorrowland_Speedway-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563164-Reviews-or60-Tomorrowland_Speedway-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563164-Reviews-or70-Tomorrowland_Speedway-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563164-Reviews-or80-Tomorrowland_Speedway-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563164-Reviews-or90-Tomorrowland_Speedway-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563164-Reviews-or100-Tomorrowland_Speedway-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563164-Reviews-or110-Tomorrowland_Speedway-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or10-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or20-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or30-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or40-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or50-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or60-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or70-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or80-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or90-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or100-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or110-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or120-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or130-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or140-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or150-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563163-Reviews-or160-Tomorrowland_Transit_Authority_PeopleMover-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or10-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or20-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or30-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or40-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or50-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or60-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or70-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or80-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or90-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or100-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or110-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or120-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or130-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or140-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or150-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or160-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or170-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or180-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or190-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or200-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or210-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or220-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or230-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or240-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or250-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or260-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or270-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or280-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or290-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or300-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563492-Reviews-or310-Toy_Story_Midway_Mania-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563278-Reviews-TriceraTop_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563278-Reviews-or10-TriceraTop_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563278-Reviews-or20-TriceraTop_Spin-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or10-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or20-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or30-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or40-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or50-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or60-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or70-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or80-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or90-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or100-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or110-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or120-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or130-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or140-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or150-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or160-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563167-Reviews-or170-Under_the_Sea_Journey_of_The_Little_Mermaid-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563172-Reviews-Walt_Disney_World_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563172-Reviews-or10-Walt_Disney_World_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563172-Reviews-or20-Walt_Disney_World_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563172-Reviews-or30-Walt_Disney_World_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563172-Reviews-or40-Walt_Disney_World_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563172-Reviews-or50-Walt_Disney_World_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563172-Reviews-or60-Walt_Disney_World_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563172-Reviews-or70-Walt_Disney_World_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563172-Reviews-or80-Walt_Disney_World_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563172-Reviews-or90-Walt_Disney_World_Railroad-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-or10-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-or20-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-or30-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-or40-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-or50-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-or60-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-or70-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-or80-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-or90-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-or100-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-or110-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-or120-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-or130-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-or140-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563168-Reviews-or150-Walt_Disney_s_Carousel_of_Progress-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563283-Reviews-Wildlife_Express_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563283-Reviews-or10-Wildlife_Express_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563283-Reviews-or20-Wildlife_Express_Train-Orlando_Florida.html',
'https://www.tripadvisor.com/Attraction_Review-g34515-d8563283-Reviews-or30-Wildlife_Express_Train-Orlando_Florida.html'
]

In [ ]:
len(URL)

for url in range(0,960):
    req = requests.get(URL[url])

sleep(randint(2,10))

In [ ]:
soup = bs(req.text, 'html.parser')

In [ ]:
ride = []
for name in soup.find_all('h1', {'class':'WlYyy'}):
    ride.append(name.text.strip())
ride

In [ ]:
title = []
for title in soup.find_all('div', {'class':'WlYyy cPsXC bLFSo cspKb dTqpp'}):
    title.append(title.text.strip())
title

In [ ]:
rating = []
for rating in soup.find_all('span', {'class':'ui_bubble_rating bubble_\d*'}):
    rating.append(rating.text.strip())
rating

In [ ]:
review = []
for name in soup.findAll('div', {'class':'duhwe'}):
    review.append(name.text.strip())

review

In [ ]:
for page in range(0,2):
    req = requests.get(URL[page])
    soup = bs(req.text, 'html.parser')
  
    ride = []
    for name in soup.find_all('h1', {'class':'WlYyy'}):
        ride.append(name.text.strip())
    ride
    
    title = []
    for title in soup.find_all('div', {'class':'WlYyy cPsXC bLFSo cspKb dTqpp'}):
        title.append(title.text.strip())
    title
    
    rating = []
    for rating in soup.find_all('span', {'class':'ui_bubble_rating bubble_\d*'}):
        rating.append(rating.text.strip())
    rating
    
    review = []
    for review in soup.findAll('div', {'class':'duhwe'}):
        review.append(review.text.strip())
    review
  
    sleep(randint(2,6))

### A failed attempt to try to see if I could find a way to pull the specific content I wanted that could be applied across all pages since the HTML divisions seemed to be unique to each page.

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import re

In [ ]:
url = "https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-Star_Tours_The_Adventures_Continue-Orlando_Florida.html"

In [ ]:
response = requests.get(url)

In [ ]:
data = bs(response.text, "html.parser")
print(data.title.text)

In [ ]:
review_data = data.find_all('div', attrs={'class':'reviews-tab'})
for review in review_data:
    namareview = review.findNext('a', attrs={'class':'ui_header_link social-member-event-MemberEventOnObjectBlock__member--35-jC'})[:].text.strip()
    bubblereview = data.find_all('span', {'class': re.compile('ui_bubble_rating bubble_\d*')})
    ratings = re.findall('\d+', ''.join(map(str, bubblereview)))
    titlereview = data.find_all('a', attrs={'class':'location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT'})[:].text.strip()
    print (namareview,ratings,titlereview)

In [ ]:
ride_review_1 = requests.get("https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-Star_Tours_The_Adventures_Continue-Orlando_Florida.html")
ride_review_1.status_code

In [ ]:
soup = bs(ride_review_1.content, 'lxml')

In [ ]:
ride_1 = []
for name in soup.findAll('div', {'class':'listing_title'}):
    ride_1.append(name.text.strip())

ride_1

### My earliest attempt at scraping the data. Looks like some of it got deleted as I was playing around with various options.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
from bs4 import BeautifulSoup as bs

In [ ]:
test_url = "https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-Star_Tours_The_Adventures_Continue-Orlando_Florida.html"

Page 1 URL: https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-Star_Tours_The_Adventures_Continue-Orlando_Florida.html
<br>Page 2 URL: https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or10-Star_Tours_The_Adventures_Continue-Orlando_Florida.html
<br>Page 3 URL: https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or20-Star_Tours_The_Adventures_Continue-Orlando_Florida.html
<br>Page 4 URL: https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or30-Star_Tours_The_Adventures_Continue-Orlando_Florida.html
        
Each page has 10 reviews, and there are 210 reviews, so we need 21 pages.

In [ ]:
ride_review

In [ ]:
ride_review.text

In [ ]:
soup = bs(ride_review.content, 'html.parser')

In [ ]:
print(soup.prettify())

In [ ]:
review_div = soup.findAll(name = "div", attrs = {"class": "dDwZb"})

In [ ]:
review_div

In [ ]:
review_div_2 = soup.findAll(name = "div", attrs = {"class": "cjhIj"})

In [ ]:
review_div_2

In [ ]:
review_div_3 = soup.findAll(name = "div", attrs = {"class": "fCFzN"})

In [ ]:
review_div_3

In [ ]:
review_div_4 = soup.findAll(name = "div", attrs = {"class": "duhwe"})

In [ ]:
review_div_4

In [ ]:
review_div_5 = soup.findAll(name = "svg", attrs = {"class": "RWYkj d H0"})

In [ ]:
review_div_5

In [ ]:
len(review_div_5)

In [ ]:
review_div_6

In [ ]:
ride_review_div = soup.findAll(name = "div", attrs = {"aria-label=": " of 5 bubbles"})

In [ ]:
ride_review_div

### A failed attempt at a for loop

In [ ]:
for review in soup.findAll('a',{'class':'review_count'}):
  a = review['href']
  a = 'https://www.tripadvisor.'+ a
  a = a[:(a.find('Reviews')+7)] + '-or{}' + a[(a.find('Reviews')+7):]
  print(a)
  links.append(a)

In [ ]:
for i in range(0,200,10):
     url = (f'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or{0}-Star_Tours_The_Adventures_Continue-Orlando_Florida.html')
     page = requests.get(url)
     soup = BeautifulSoup(page.content, 'html.parser')
     results = soup.find('div', { 'id' : 'cmp-container'})
     elems = results.find_all(class_='cmp-Review-container')
     for elem in elems:
         title = elem.find(attrs = {'class':'cmp-Review-title'})
         review = elem.find('div', {'class': 'cmp-Review-text'})
         author = elem.find(attrs = {'class':'cmp-Review-author'})
         rating = elem.find(attrs = {'class':'cmp-ReviewRating-text'})
         df = df.append({'review_title': title.text,
          'review': review.text,
        'author': author.text,
          'rating': rating.text
            }, ignore_index=True)

In [ ]:
<svg class="RWYkj d H0" viewBox="0 0 88 16" width="88" height="16" aria-label="5.0 of 5 bubbles" title="5.0 of 5 bubbles"><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(18 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(36 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(54 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(72 0) scale(0.6666666666666666)"></path></svg>

In [ ]:
<div class="RTVWf o W f u w eeCyE" aria-label="4.5 of 5 bubbles. 306 reviews"><svg class="RWYkj d H0" viewBox="0 0 88 16" width="88" height="16" aria-label="" title=""><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(18 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(36 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(54 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(72 0) scale(0.6666666666666666)"></path></svg><span class="WlYyy diXIH bGusc dDKKM"><span class="cfIVb">306</span></span></div>

### A failed attempt at using selenium. Maybe it was just a Windows thing and iOS would have worked?

In [ ]:
!pip install selenium

In [ ]:
import sys
import csv
from selenium import webdriver
import time

# default path to file to store data
path_to_file = "/Users/Christopher.Doughty/Downloads/Data Science/Notebooks/project eda brief/Scraping-TripAdvisor-with-Python-2020/ridereviews.csv"

# default number of scraped pages
num_page = 20

# default tripadvisor website of restaurant
url = "https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or30-Star_Tours_The_Adventures_Continue-Orlando_Florida.html"

# if you pass the inputs in the command line
if (len(sys.argv) == 4):
    path_to_file = sys.argv[1]
    num_page = int(sys.argv[2])
    url = sys.argv[3]

# Import the webdriver
driver = webdriver.Firefox()
driver.get(url)

# Open the file to save the review
csvFile = open(path_to_file, 'a', encoding="utf-8")
csvWriter = csv.writer(csvFile)

# change the value inside the range to save more or less reviews
for i in range(0, num_page):
    
    # expand the review 
    time.sleep(2)
    driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']").click()

    container = driver.find_elements_by_xpath(".//div[@class='review-container']")

    for j in range(len(container)):

        title = container[j].find_element_by_xpath(".//span[@class='noQuotes']").text
        date = container[j].find_element_by_xpath(".//span[contains(@class, 'ratingDate')]").get_attribute("title")
        rating = container[j].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
        review = container[j].find_element_by_xpath(".//p[@class='partial_entry']").text.replace("\n", " ")

        csvWriter.writerow([date, rating, title, review]) 

    # change the page
    driver.find_element_by_xpath('.//a[@class="nav next ui_button primary"]').click()

driver.close()

### A failed attempt at converting to json for processing.

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import json




url = "https://www.tripadvisor.com/Attraction_Review-g34515-d8563101-Reviews-Astro_Orbiter-Orlando_Florida.html"
response = requests.get(url)
data = bs(response.text, "html.parser")
print(data.title.text)

nama = data.find('h1', attrs={'class':'ui_header h1'})[0].text.strip()
print (nama)

category = data.find('div', attrs={'class':'attractions-attraction-review-header-AttractionLinks__detail--2-xvX'})[0].text.strip()
print (category)

location= data.find('div', attrs={'class':'attractions-contact-card-ContactCard__contactRow--3Ih6v'})[0].text.strip()
print (location)

# Get Total count of reviews
data = BeautifulSoup(response.text, "html.parser")

reviewDataIDs = []
scripts = data.find_all('script')
for script in scripts:
    if 'window.__WEB_CONTEXT__=' in script.text:
        jsonStr = script.text
        jsonStr = jsonStr.split('window.__WEB_CONTEXT__={pageManifest:')[-1]

        iterateJson = True
        while iterateJson == True:
            try:
                jsonData = json.loads(jsonStr + '}')
                iterateJson = False
            except:
                jsonStr = jsonStr.rsplit('}',1)[0]


raiseError = True
for k, v in jsonData['urqlCache'].items():
    try:
        totalCount = jsonData['urqlCache'][k]['data']['locations'][0]['reviewListPage']['totalCount']  
        raiseError = False
        reviewDataIDs.append(k)
        break
    except:
        pass




def getJsonData(reviewCount, reviewDataIDs, continueLoop):
    while continueLoop == True:
        url = "https://www.tripadvisor.co.uk/Attraction_Review-g297722-d6611509-Reviews-or%s-Pahawang_Island-Bandar_Lampung_Lampung_Sumatra.html#REVIEWS"  %reviewCount   
        response = requests.get(url, headers=headers)
        data = BeautifulSoup(response.text, "html.parser")

        scripts = data.find_all('script')
        for script in scripts:
            if 'window.__WEB_CONTEXT__=' in script.text:
                jsonStr = script.text
                jsonStr = jsonStr.split('window.__WEB_CONTEXT__={pageManifest:')[-1]

                iterateJson = True
                while iterateJson == True:
                    try:
                        jsonData = json.loads(jsonStr + '}')
                        iterateJson = False
                    except:
                        jsonStr = jsonStr.rsplit('}',1)[0]


        raiseError = True
        for k, v in jsonData['urqlCache'].items():
            try:
                reviewData = jsonData['urqlCache'][k]['data']['locations'][0]['reviewListPage']['reviews']  
                raiseError = False
                if k not in reviewDataIDs:#
                    continueLoop = False
                    reviewDataIDs.append(k)
                break
            except:
                pass

        if raiseError == True:
            raise ValueError ('Data could not be found.')

        if continueLoop == False:
            return reviewData, reviewDataIDs



# Get Reviews
for reviewCount in list(range(0,totalCount,10)):
    reviewData, reviewDataIDs = getJsonData(reviewCount, reviewDataIDs, continueLoop=True)

    for each in reviewData:
        ride = data.title.text
        rating = each['rating']
        title = each['title']
        text = each['text']
        user = each['username']

        print  ('Ride: %s\nName: %s\nTitle: %s\nRating: %s\nReview: %s\n' %(ride, user, title, rating, text) + '-'*70 + '\n')

### A failed attempt at using scrapy to scrape the data.

In [1]:
import scrapy
from..items import ReviewItem
import re


class TripAdvisorReview(scrapy.Spider):
    name = "tripadvisor"
    start_urls = ["https://www.tripadvisor.co.uk/Hotels-g186217-England-Hotels.html"]

    def parse(self, response):
        for href in response.css("div.listing_title a::attr('href')"):
            url = response.urljoin(href.extract())
            yield scrapy.Request(url, callback=self.parse_hotel)

    def parse_hotel(self, response):
        for info in response.css('div.page'):
            items = ReviewItem()
            hotel_names = info.css('._1mTlpMC3::text').extract()
            hotel_names = [hotel_name.strip() for hotel_name in hotel_names]
            reviewer_names = info.css('._1r_My98y::text').extract()
            reviewer_names = [reviewer_name.strip() for reviewer_name in reviewer_names]
            reviewer_contributions = info.css('._3fPsSAYi:nth-child(1) ._1fk70GUn , ._1TuWwpYf+ ._3fPsSAYi ._1fk70GUn').css('::text').extract()
            reviewer_contributions = [reviewer_contribution.strip() for reviewer_contribution in reviewer_contributions]
            review_dates = info.xpath('//div[@class = "_2fxQ4TOx"]/span[contains(text(),"wrote a review")]/text()').extract()
            review_dates = [review_date.strip() for review_date in review_dates]
            review_stars = info.css('div.nf9vGX55 .ui_bubble_rating').xpath("@class").extract()
            review_stars = [review_star.strip() for review_star in review_stars]
            review_texts = info.css('#component_15 .cPQsENeY').css('::text').extract()
            review_texts = [review_text.strip() for review_text in review_texts]
            #helpful_vote = info.css('._3kbymg8R::text').extract()
            result = zip(hotel_names, reviewer_names, review_dates, review_texts, review_stars, reviewer_contributions)

        for hotel_name, reviewer_name, review_date, review_text, review_star, reviewer_contribution in result:
            items['hotel_name'] = hotel_name
            items['reviewer_name'] = reviewer_name
            items['reviewer_contribution'] = reviewer_contribution
            items['review_date'] = review_date
            items['review_star'] = review_star
            items['review_text'] = review_text
            #items['helpful_vote'] = helpful_vote
            yield items

ImportError: attempted relative import with no known parent package

In [1]:
import scrapy
#from ..items import ReviewItem

class TripAdvisorReview(scrapy.Spider):
    
    name = "tripadvisor"
    start_urls = ["https://www.tripadvisor.co.uk/Hotels-g186217-England-Hotels.html"]

    def parse(self, response):
        for href in response.css("div.listing_title a::attr('href')"):
            url = response.urljoin(href.extract())
            yield scrapy.Request(url, callback=self.parse_hotel)

    def parse_hotel(self, response):
        for info in response.css('div.page'):
            hotel_names = info.css('._1mTlpMC3::text').extract()
            hotel_names = [hotel_name.strip() for hotel_name in hotel_names]
            
            reviewer_names = info.css('._1r_My98y::text').extract()
            reviewer_names = [reviewer_name.strip() for reviewer_name in reviewer_names]
            reviewer_contributions = info.css('._3fPsSAYi:nth-child(1) ._1fk70GUn , ._1TuWwpYf+ ._3fPsSAYi ._1fk70GUn').css('::text').extract()
            reviewer_contributions = [reviewer_contribution.strip() for reviewer_contribution in reviewer_contributions]
            review_dates = info.xpath('//div[@class = "_2fxQ4TOx"]/span[contains(text(),"wrote a review")]/text()').extract()
            review_dates = [review_date.strip() for review_date in review_dates]
            review_stars = info.css('div.nf9vGX55 .ui_bubble_rating').xpath("@class").extract()
            review_stars = [review_star.strip() for review_star in review_stars]
            review_texts = info.css('#component_15 .cPQsENeY').css('::text').extract()
            review_texts = [review_text.strip() for review_text in review_texts]
            #helpful_vote = info.css('._3kbymg8R::text').extract()

            print('hotel_names:', len(hotel_names))
            print('reviewer_names:', len(reviewer_names))
            print('review_dates:', len(review_dates))
            print('review_stars:', len(review_stars))
            print('review_texts:', len(review_texts))
            print('reviewer_contributions:', len(reviewer_contributions))
            print('----')
            
            # without `hotel_names`
            all_reviews = zip(reviewer_names, review_dates, review_texts, review_stars, reviewer_contributions)

            hotel_name = hotel_names[0]  # <-- manually get first hotel

            # without `hotel_name`
            for reviewer_name, review_date, review_text, review_star, reviewer_contribution in all_reviews:
                #items = ReviewItem()
                items = dict()
                items['hotel_name'] = hotel_name  # <-- manually add first hotel
                items['reviewer_name'] = reviewer_name
                items['reviewer_contribution'] = reviewer_contribution
                items['review_date'] = review_date
                items['review_star'] = review_star
                items['review_text'] = review_text
                #items['helpful_vote'] = helpful_vote
                yield items
            
# --- run without project and save in `output.csv` ---

from scrapy.crawler import CrawlerProcess

c = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0',
    # save in file CSV, JSON or XML
    'FEED_FORMAT': 'csv',     # csv, json, xml
    'FEED_URI': 'output.csv', #
})
c.crawl(TripAdvisorReview)
c.start()

2021-10-08 20:14:48 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-10-08 20:14:48 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.7, Platform Windows-10-10.0.16299-SP0
2021-10-08 20:14:48 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-10-08 20:14:48 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/5.0'}
2021-10-08 20:14:48 [scrapy.extensions.telnet] INFO: Telnet Password: 04ba2927bb68fba3
2021-10-08 20:14:48 [py.warnings] WARNING: C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\extensions\feedexport.py:247: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more de

hotel_names: 0
reviewer_names: 0
review_dates: 0
review_stars: 0
review_texts: 0
reviewer_contributions: 0
----
hotel_names: 0
reviewer_names: 0
review_dates: 0
review_stars: 0
review_texts: 0
reviewer_contributions: 0
----
hotel_names: 0
reviewer_names: 0
review_dates: 0
review_stars: 0
review_texts: 0
reviewer_contributions: 0
----
hotel_names: 0
reviewer_names: 0
review_dates: 0
review_stars: 0
review_texts: 0
reviewer_contributions: 0
----
hotel_names: 0
reviewer_names: 0
review_dates: 0
review_stars: 0
review_texts: 0
reviewer_contributions: 0
----


2021-10-08 20:14:52 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.tripadvisor.co.uk/Hotel_Review-g186338-d193121-Reviews-The_Milestone_Hotel_and_Residences-London_England.html> (referer: https://www.tripadvisor.co.uk/Hotels-g186217-England-Hotels.html)
Traceback (most recent call last):
  File "C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\utils\defer.py", line 120, in iter_errback
    yield next(it)
  File "C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\utils\python.py", line 353, in __next__
    return next(self.data)
  File "C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\utils\python.py", line 353, in __next__
    return next(self.data)
  File "C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\core\spidermw.py", line 56, in _evaluate_iterable
    for r in iterable:
  File "C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\spidermiddlewares\offsite.py", line 29, in process_spider_o

hotel_names: 0
reviewer_names: 0
review_dates: 0
review_stars: 0
review_texts: 0
reviewer_contributions: 0
----
hotel_names: 0
reviewer_names: 0
review_dates: 0
review_stars: 0
review_texts: 0
reviewer_contributions: 0
----
hotel_names: 0
reviewer_names: 0
review_dates: 0
review_stars: 0
review_texts: 0
reviewer_contributions: 0
----


2021-10-08 20:14:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.tripadvisor.co.uk/Hotel_Review-g503998-d1199828-Reviews-All_Seasons_Guest_House_Filey_B_B-Filey_Scarborough_District_North_Yorkshire_England.html> (referer: https://www.tripadvisor.co.uk/Hotels-g186217-England-Hotels.html)
2021-10-08 20:14:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.tripadvisor.co.uk/Hotel_Review-g186366-d12440508-Reviews-Heritage_View_Guest_House-Ironbridge_Ironbridge_Gorge_Telford_Shropshire_England.html> (referer: https://www.tripadvisor.co.uk/Hotels-g186217-England-Hotels.html)
2021-10-08 20:14:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.tripadvisor.co.uk/Hotel_Review-g504012-d1090679-Reviews-Raven_House_Bed_Breakfast-Robin_Hoods_Bay_North_York_Moors_National_Park_North_Yorkshi.html> (referer: https://www.tripadvisor.co.uk/Hotels-g186217-England-Hotels.html)
2021-10-08 20:14:52 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.tripa

...

hotel_names: 0
reviewer_names: 0
review_dates: 0
review_stars: 0
review_texts: 0
reviewer_contributions: 0
----
hotel_names: 0
reviewer_names: 0
review_dates: 0
review_stars: 0
review_texts: 0
reviewer_contributions: 0
----


2021-10-08 20:14:54 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.tripadvisor.co.uk/Hotel_Review-g187069-d15879427-Reviews-Dakota_Manchester-Manchester_Greater_Manchester_England.html> (referer: https://www.tripadvisor.co.uk/Hotels-g186217-England-Hotels.html)
Traceback (most recent call last):
  File "C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\utils\defer.py", line 120, in iter_errback
    yield next(it)
  File "C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\utils\python.py", line 353, in __next__
    return next(self.data)
  File "C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\utils\python.py", line 353, in __next__
    return next(self.data)
  File "C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\core\spidermw.py", line 56, in _evaluate_iterable
    for r in iterable:
  File "C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\spidermiddlewares\offsite.py", line 29, in process_

hotel_names: 0
reviewer_names: 0
review_dates: 0
review_stars: 0
review_texts: 0
reviewer_contributions: 0
----
hotel_names: 0
reviewer_names: 0
review_dates: 0
review_stars: 0
review_texts: 0
reviewer_contributions: 0
----


2021-10-08 20:14:54 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.tripadvisor.co.uk/Hotel_Review-g186338-d8625676-Reviews-The_Resident_Victoria-London_England.html> (referer: https://www.tripadvisor.co.uk/Hotels-g186217-England-Hotels.html)
Traceback (most recent call last):
  File "C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\utils\defer.py", line 120, in iter_errback
    yield next(it)
  File "C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\utils\python.py", line 353, in __next__
    return next(self.data)
  File "C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\utils\python.py", line 353, in __next__
    return next(self.data)
  File "C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\core\spidermw.py", line 56, in _evaluate_iterable
    for r in iterable:
  File "C:\Users\Christopher.Doughty\Anaconda3\lib\site-packages\scrapy\spidermiddlewares\offsite.py", line 29, in process_spider_output
    fo

hotel_names: 0
reviewer_names: 0
review_dates: 0
review_stars: 0
review_texts: 0
reviewer_contributions: 0
----


In [7]:
pip install scrapy

  Created wheel for protego: filename=Protego-0.1.16-py3-none-any.whl size=7764 sha256=641dea5d33ad178b9f431ec2d29fcf7e45d8f13f32283dc4e32b6424ec13d1ed
  Stored in directory: c:\users\christopher.doughty\appdata\local\pip\cache\wheels\91\64\36\bd0d11306cb22a78c7f53d603c7eb74ebb6c211703bc40b686
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=12547 sha256=7d4845a7b8e7caafbb539a241d4906bc543741ce6764c999d777300fb9b12090
  Stored in directory: c:\users\christopher.doughty\appdata\local\pip\cache\wheels\3c\31\7f\d7d7b5f0b9bad841ed856138ff0c5ee2bf2e04dbeb413097c8
Successfully built protego PyDispatcher
Note: you may need to restart the kernel to use updated packages.


In [3]:
import scrapy

In [4]:
#from ..items import ReviewItem

class TripAdvisorReview(scrapy.Spider):
    
    name = "tripadvisor"
    start_urls = ["https://www.tripadvisor.co.uk/Hotels-g186217-England-Hotels.html"]

    def parse(self, response):
        for href in response.css("div.listing_title a::attr('href')"):
            url = response.urljoin(href.extract())
            yield scrapy.Request(url, callback=self.parse_hotel)

    def parse_hotel(self, response):
        hotel_name = response.css('#HEADING::text').extract_first().strip()
        print('hotel_name:', hotel_name)
        
        for review in response.xpath('.//div[@data-test-target="HR_CC_CARD"]'):
            name = review.css('._1r_My98y::text').extract_first().strip()
            contribution = review.css('._3fPsSAYi:nth-child(1) ._1fk70GUn , ._1TuWwpYf+ ._3fPsSAYi ._1fk70GUn').css('::text').extract_first().strip()
            date = review.xpath('.//div[@class="_2fxQ4TOx"]/span[contains(text(),"wrote a review")]/text()').extract_first().strip().replace('wrote a review ', '')
            stars = review.css('div.nf9vGX55 .ui_bubble_rating').xpath("@class").extract_first().strip().replace('ui_bubble_rating bubble_', '')
            text = review.xpath('.//div[@class="cPQsENeY"]//span/text()').extract_first().strip()

            #items = ReviewItem()
            items = dict()
            items['hotel_name'] = hotel_name  # <-- manually add first hotel
            items['reviewer_name'] = name
            items['reviewer_contribution'] = contribution
            items['review_date'] = date
            items['review_star'] = stars
            items['review_text'] = text

            yield items

In [5]:
print('hotel_name')

hotel_name


In [11]:
# --- run without project and save in `output.csv` ---

from scrapy.crawler import CrawlerProcess

In [14]:
c = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0',
    # save in file CSV, JSON or XML
    'FEED_FORMAT': 'csv',     # csv, json, xml
    'FEED_URI': 'output.csv', #
})
c.crawl(TripAdvisorReview)
c.start()

2021-10-08 20:10:20 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-10-08 20:10:20 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.7, Platform Windows-10-10.0.16299-SP0
2021-10-08 20:10:20 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-10-08 20:10:20 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/5.0'}
2021-10-08 20:10:20 [scrapy.extensions.telnet] INFO: Telnet Password: 34f29d3f22d57477
2021-10-08 20:10:20 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-10-08 20:10:20 [scrapy.middleware] INFO: Enabled downloader midd

ReactorNotRestartable: 

### Another attempt at using Apify.

I would often copy/paste code that semi-worked before priodically throughout to see if I could tweak it to get it to work.

In [1]:
pip install apify-client

Note: you may need to restart the kernel to use updated packages.


In [2]:
from apify_client import ApifyClient

# Initialize the ApifyClient with your API token
client = ApifyClient("apify_api_R79VT2EoA2dY5vreyEz4hEdHcjcJys2kBI1L")

# Prepare the actor input
run_input = {
    "locationFullName": "Prague",
    "maxReviews": 0,
    "language": "en",
    "currency": "USD",
    "proxyConfiguration": { "useApifyProxy": True },
}

# Run the actor and wait for it to finish
run = client.actor("maxcopell/tripadvisor").call(run_input=run_input)

# Fetch and print actor results from the run's dataset (if there are any)
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    print(item)

ApifyApiError: By launching this job you would exceed the memory limit of 4096MB for all your actors (currently used: 4096MB, requested: 4096MB). Please upgrade to a paid plan to increase your actor memory limit.

In [1]:
from apify_client import ApifyClient

In [2]:
# Initialize the ApifyClient with your API token
client = ApifyClient("apify_api_R79VT2EoA2dY5vreyEz4hEdHcjcJys2kBI1L")

In [3]:
# Prepare the actor input
run_input = {
   "locationFullName": "Orlando",
   "includeRestaurants": False,
   "includeHotels": False,
   "includeReviews": True,
   "includeAttractions": True,
   "lastReviewDate": "2010-01-01"
 }

In [4]:
# Run the actor and wait for it to finish
run = client.actor("maxcopell/tripadvisor").call(run_input=run_input)

ApifyApiError: By launching this job you would exceed the memory limit of 4096MB for all your actors (currently used: 4096MB, requested: 4096MB). Please upgrade to a paid plan to increase your actor memory limit.

In [16]:
# Fetch and print actor results from the run's dataset (if there are any)
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    print(item)

In [ ]:
{
  "locationFullName": "Orlando",
  "includeRestaurants": False,
  "includeHotels": False,
  "includeAttractions": True,
  "includeTags": False,
  "includeReviews": True,
  "lastReviewDate": "2010-01-01",
  "language": "en",
  "currency": "USD",
  "proxyConfiguration": {
    "useApifyProxy": False
  },
  "debugLog": False
}

### Another failed attempt at converting to json for processing.

In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import json


url = "https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-Star_Tours_The_Adventures_Continue-Orlando_Florida.html"
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
response = requests.get(url, headers=headers)
data = BeautifulSoup(response.text, "html.parser")
nama = data.find_all('h1', attrs={'class':'WlYyy'})[0].text.strip()
print (nama)


reviewDataIDs = []
scripts = data.find_all('script')
for script in scripts:
    if 'window.__WEB_CONTEXT__=' in script.text:
        jsonStr = script.text
        jsonStr = jsonStr.split('window.__WEB_CONTEXT__={pageManifest:')[-1]

        iterateJson = True
        while iterateJson == True:
            try:
                jsonData = json.loads(jsonStr + '}')
                iterateJson = False
            except:
                jsonStr = jsonStr.rsplit('}',1)[0]


raiseError = True
for k, v in jsonData['urqlCache'].items():
    try:
        totalCount = jsonData['urqlCache'][k]['data']['locations'][0]['reviewListPage']['totalCount']  
        raiseError = False
        reviewDataIDs.append(k)
        break
    except:
        pass


def getJsonData(reviewCount, reviewDataIDs, continueLoop):
    while continueLoop == True:
        url = "https://www.tripadvisor.co.uk/Attraction_Review-g297722-d6611509-Reviews-or%s-Pahawang_Island-Bandar_Lampung_Lampung_Sumatra.html#REVIEWS"  %reviewCount   
        response = requests.get(url, headers=headers)
        data = BeautifulSoup(response.text, "html.parser")

        scripts = data.find_all('script')
        for script in scripts:
            if 'window.__WEB_CONTEXT__=' in script.text:
                jsonStr = script.text
                jsonStr = jsonStr.split('window.__WEB_CONTEXT__={pageManifest:')[-1]

                iterateJson = True
                while iterateJson == True:
                    try:
                        jsonData = json.loads(jsonStr + '}')
                        iterateJson = False
                    except:
                        jsonStr = jsonStr.rsplit('}',1)[0]


        raiseError = True
        for k, v in jsonData['urqlCache'].items():
            try:
                reviewData = jsonData['urqlCache'][k]['data']['locations'][0]['reviewListPage']['reviews']  
                raiseError = False
                if k not in reviewDataIDs:#
                    continueLoop = False
                    reviewDataIDs.append(k)
                break
            except:
                pass

        if raiseError == True:
            raise ValueError ('Data could not be found.')

        if continueLoop == False:
            return reviewData, reviewDataIDs


# Get Reviews
for reviewCount in list(range(0,totalCount,5)):
    reviewData, reviewDataIDs = getJsonData(reviewCount, reviewDataIDs, continueLoop=True)

    for each in reviewData:
        rating = each['rating']
        title = each['title']
        text = each['text']
        user = each['username']

        print  ('Name: %s\nTitle: %s\nRating: %s\nReview: %s\n' %(user, title, rating, text) + '-'*70 + '\n')

Star Tours - The Adventures Continue


NameError: name 'jsonData' is not defined

### A failed attempt at a for loop.

In [ ]:
url = "https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-Star_Tours_The_Adventures_Continue-Orlando_Florida.html"
response = requests.get(url)
data = bs(response.text, "html.parser")
print(data.title.text)

nama = data.find_all('h1', attrs={'class':'WlYyy'})[:].text.strip()
print (nama)

category = data.find_all('div', attrs={'class':'attractions-attraction-review-header-AttractionLinks__detail--2-xvX'})[:].text.strip()
print (category)

location= data.find_all('div', attrs={'class':'attractions-contact-card-ContactCard__contactRow--3Ih6v'})[:].text.strip()
print (location)

review_data = data.find_all('div', attrs={'class':'reviews-tab'})
for review in review_data:
    namareview = review.findNext('a', attrs={'class':'ui_header_link social-member-event-MemberEventOnObjectBlock__member--35-jC'})[:].text.strip()
    bubblereview = data.find_all('span', {'class': re.compile('ui_bubble_rating bubble_\d*')})
    ratings = re.findall('\d+', ''.join(map(str, bubblereview)))
    titlereview = data.find_all('a', attrs={'class':'location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT'})[:].text.strip()
    print (namareview,ratings,titlereview)
    
    sleep(randint(2,10))

### A copy of my earliest scraping data code. I got pretty good at scraping the data I wanted for a specific page, but unfortunately, the classes and divisions I was using to isolate the portions I wanted were not shared site-wide.

In [2]:
ride_review_1 = requests.get("https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-Star_Tours_The_Adventures_Continue-Orlando_Florida.html")

In [3]:
ride_review_1.status_code

200

In [4]:
soup = bs(ride_review_1.content, 'html.parser')

In [5]:
review_1_name = soup.findAll(name = "div", attrs = {"class": "MmEaS"})

In [6]:
review_1_name

[<div class="MmEaS"><h1 class="WlYyy cPsXC GeSzT" data-automation="mainH1">Star Tours - The Adventures Continue</h1></div>]

In [7]:
review_1_name = []
for name in soup.findAll('h1', {'class':'WlYyy'}):
    review_1_name.append(name.text.strip())

review_1_name

['Star Tours - The Adventures Continue']

In [8]:
review_1_group = soup.findAll(name = "div", attrs = {"class": "bPhtn"})

In [9]:
review_1_group

[<div class="bPhtn"><div><span data-ft="true"><span data-ft="true"><div class="dDwZb f M k"><div class="bJyQA f u o"><div class="fksET"><div class="baYaB f u"><div class="bQEAh" style="z-index:0"><div class="svvjk Rb I o fknJY"><a aria-hidden="true" class="iPqaD _F G- ddFHE eKwUx" href="/Profile/theeJG" tabindex="-1" target="_self"><div class="btBEK eglJy"><picture class="dugSS _R" style="width:32px;height:32px"><img alt="Jared" height="100" loading="lazy" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/f6/e4/2d/default-avatar-2020-48.jpg?w=100&amp;h=-1&amp;s=1" width="100"/></picture></div></a></div></div></div></div><div class="cjhIj"><span class="WlYyy cPsXC dTqpp"><a class="iPqaD _F G- ddFHE eKwUx btBEK fUpii" href="/Profile/theeJG" target="_self">Jared</a></span><div class="ddOtn"><div class="WlYyy diXIH bQCoY"><span>Uniontown, PA</span><span class="bjxnD">26 contributions</span></div></div></div></div><div class="euzOH f"><button aria-label="Click to add helpful v

In [10]:
review_1_group = []
for review in soup.findAll('div', {'class':'bPhtn'}):
    review_1_group.append(name.text.strip())

review_1_group

['Star Tours - The Adventures Continue']

In [11]:
review_1_stars = soup.findAll(name = "div", attrs = {"class": "fCFzN"})

In [12]:
review_1_stars

[<div class="fCFzN"></div>,
 <div class="fCFzN"></div>,
 <div class="fCFzN"></div>,
 <div class="fCFzN"></div>,
 <div class="fCFzN"></div>,
 <div class="fCFzN"></div>,
 <div class="fCFzN"></div>,
 <div class="fCFzN"></div>,
 <div class="fCFzN"></div>,
 <div class="fCFzN"></div>]

In [13]:
review_1_stars = soup.findAll(name = "svg", attrs = {"class": "RWYkj d H0"})

In [14]:
review_1_stars

[<svg aria-label="" class="RWYkj d H0" height="16" title="" viewbox="0 0 88 16" width="88"><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(18 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(36 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(54 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(72 0) scale(0.6666666666666666)"></path></svg>,
 <svg aria-label="5.0 of 5 bubbles" class="RWYkj d H0" height="12" title="5.0 of 5 bubbles" viewbox="0 0 

In [12]:
ratings = []
for rating in soup.find_all('span', {'class':'ui_bubble_rating bubble_\d*'}):
    ratings.append(name.text.strip())

ratings

[]

In [13]:
bubblereview = soup.find_all('span', {'class': re.compile('ui_bubble_rating bubble_\d*')})
ratings = re.findall('\d+', ''.join(map(str, bubblereview)))

In [14]:
ratings

[]

In [18]:
rating = []
for rating in soup.find_all('span', {'class':'ui_bubble_rating bubble_\d*'}):
    rating.append(rating.text.strip())

rating

[]

In [16]:
review_1_stars = soup.findAll(name = "fCFzN", attrs = {"label=": "bubbles"})

In [17]:
review_1_stars

[]

In [18]:
review_1_stars = []
for stars in soup.findAll('a', {'class':'ui_bubble_rating'}):
    review_1_stars.append(stars['alt'])

review_1_stars

[]

In [19]:
review_1_text = soup.findAll(name = "div", attrs = {"class": "duhwe"})

In [20]:
review_1_text

[<div class="duhwe _T bOlcm dMbup"><div class="pIRBV _T KRIav" style="-webkit-line-clamp:7;line-break:normal;cursor:pointer"><div class="WlYyy diXIH dDKKM"><span class="NejBf">So good! The constant changing of which planets you visit helps to make this a ride we visit several times during our trip!</span></div></div><div class="eIVRK"><button class="bfQwA _G B- _S _T c G_ P0 ddFHE cnvzr" type="button"><span class="WlYyy cacGK Wb">Read more</span><div class="bLMrm"><svg class="fecdL d Vb wQMPa" height="20px" viewbox="0 0 24 24" width="20px"><path d="M18.4 7.4L12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></button></div></div>,
 <div class="duhwe _T bOlcm dMbup"><div class="pIRBV _T KRIav" style="-webkit-line-clamp:7;line-break:normal;cursor:pointer"><div class="WlYyy diXIH dDKKM"><span class="NejBf">Star Tours is the oldest Star Wars ride at Disney Hollywood studios, but it’s still a very entertaining ride. The wait time for the ride was no longer than 20 minutes during m

In [21]:
len(review_1_text)

10

In [22]:
review_1_text = []
for name in soup.findAll('div', {'class':'duhwe'}):
    review_1_text.append(name.text.strip())

review_1_text

['So good! The constant changing of which planets you visit helps to make this a ride we visit several times during our trip!Read more',
 'Star Tours is the oldest Star Wars ride at Disney Hollywood studios, but it’s still a very entertaining ride. The wait time for the ride was no longer than 20 minutes during my visit, I assume this was because the new Disney Star Wars attractions are attracting most park guests.  This is one ride  you don’t want to pass-up, you will definitely enjoy this ride, especially if you are a Star Wars fan. This is a great entertainment for the family no matter your age, and the ride will get you out of the heat if you’re visiting during the warmer parts of the year.Read more',
 'I remember how amazing this ride was when it was new.  As a die-hard SW fan, it was a dream come true.  Disney does a great job of keeping older rides fresh by updating them, and this is true of Star Tours.  However, I would love to see a bigger overhaul, with multiple missions rand

In [23]:
len(review_1_text)

10

In [24]:
review_1_nature = []
for name in soup.find_all('div', {'class':'fEDvV'}):
    review_1_nature.append(name.text.strip())

review_1_nature

['Sep 2021 • Family',
 'Jun 2021 • Family',
 'Feb 2021',
 'Dec 2020',
 'Dec 2020',
 'Dec 2020',
 'Dec 2020',
 'Nov 2020',
 'Sep 2019 • Couples',
 'Mar 2020 • Friends']

In [25]:
review_1_header = []
for name in soup.find_all('div', {'class':'WlYyy cPsXC bLFSo cspKb dTqpp'}):
    review_1_header.append(name.text.strip())

review_1_header

['Very re-rideable',
 'Great Ride!',
 'Still a really great ride!',
 'Shorter lines, still fun!',
 'Still a great ride',
 'Star Wars Fan Fare',
 'A fun ride',
 'Two Rides, Two New Tours',
 'Better every time!',
 'Love How This Ride Has Adapted Over The Years!']

In [26]:
review_1_location = []
for name in soup.find_all('div', {'class':'ddOtn'}):
    review_1_location.append(name.text.strip())

review_1_location

['Uniontown, PA26 contributions',
 'Wichita, KS2,343 contributions',
 'Castle Rock, CO2,853 contributions',
 'Austin, TX2,612 contributions',
 'Arlington, VA1,178 contributions',
 'Columbus, OH223 contributions',
 'Oregon City, OR817 contributions',
 'Simpsonville, SC639 contributions',
 'Warrington, UK161 contributions',
 'Bognor Regis, UK310 contributions',
 'Buenos Aires, Argentina86 contributions',
 '',
 'Estero, FL951 contributions',
 'Virginia Beach, VA34 contributions',
 'Barrie, Canada116 contributions',
 'Miami, FL21 contributions',
 'Tampa, FL12 contributions']

In [27]:
review_1_stars = []
for stars in soup.find_all('svg', {'class':'RWYkj d H0'}):
    review_1_stars.append(stars.text.strip())

review_1_stars

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [28]:
len(review_1_stars)

74

https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or10-Star_Tours_The_Adventures_Continue-Orlando_Florida.html

### Another failed for loop.

In [30]:
html = requests.get('https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-Star_Tours_The_Adventures_Continue-Orlando_Florida.html')
soup = bs(html.content,'lxml')

In [34]:
links = []

for review in bs.find_all('a',{'class':'review_count'}):
  a = review['href']
  a = 'https://www.tripadvisor.com'+ a
  a = a[:(a.find('Reviews')+20)] + '-or{}' + a[(a.find('Reviews')+20):]
  print(a)
  links.append(a)
links

AttributeError: 'str' object has no attribute 'descendants'

In [ ]:
from random import randint
from time import sleep

In [ ]:
reviews = []

for link in links:
  d = [10,20,30,40,50,60,70,80,90,100,110,120,140,160,180,200]
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'}
    html2 = requests.get(link.format(i for i in range(10,200,10)),headers=headers)
    sleep(randint(1,5))
    soup_2 = bs(html2.content,'lxml')
    for r in soup_2.findAll('q'):
        reviews.append(r.span.text.strip())
        print(r.span.text.strip())

reviews

### Another for loop that didn't work.

In [ ]:
for i in range(10,200,10):
     url = (f'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or{0}-Star_Tours_The_Adventures_Continue-Orlando_Florida.html')
     page = requests.get(url)
     soup = bs(page.content, 'html.parser')
     results = soup.find('div', { 'id' : 'cmp-container'})
     elems = results.find(class_='cmp-Review-container')
     for elem in elems:
         title = elem.find(attrs = {'class':'cmp-Review-title'})
         review = elem.find('div', {'class': 'cmp-Review-text'})
         author = elem.find(attrs = {'class':'cmp-Review-author'})
         rating = elem.find(attrs = {'class':'cmp-ReviewRating-text'})
         df = df.append({'review_title': title.text,
          'review': review.text,
        'author': author.text,
          'rating': rating.text
            }, ignore_index=True)

### Another for loop that didn't work.

In [ ]:
for review in soup.findAll('a',{'class':'review_count'}):
  a = review['href']
  a = 'https://www.tripadvisor.in'+ a
  a = a[:(a.find('Reviews')+7)] + '-or{}' + a[(a.find('Reviews')+7):]
  print(a)
  links.append(a)

In [4]:
soup = bs(ride_review_1.content, 'lxml')

In [5]:
ride_1 = []
for name in soup.findAll('div', {'class':'listing_title'}):
    ride_1.append(name.text.strip())

ride_1

[]

### A copy of my earliest scraping attempt.

I'd often copy/paste new starter code to build off of what worked in the past and then see if small edits would change anything.

In [102]:
test_url = "https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-Star_Tours_The_Adventures_Continue-Orlando_Florida.html"

Page 1 URL: https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-Star_Tours_The_Adventures_Continue-Orlando_Florida.html
<br>Page 2 URL: https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or10-Star_Tours_The_Adventures_Continue-Orlando_Florida.html
<br>Page 3 URL: https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or20-Star_Tours_The_Adventures_Continue-Orlando_Florida.html
<br>Page 4 URL: https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or30-Star_Tours_The_Adventures_Continue-Orlando_Florida.html
        
Each page has 10 reviews, and there are 210 reviews, so we need 21 pages.

In [104]:
ride_review

<Response [200]>

In [105]:
ride_review.text

'<!DOCTYPE html><html lang="en"><head><meta http-equiv="content-type" content="text/html; charset=utf-8"/><meta http-equiv="content-language" content="en"/><link href="https://static.tacdn.com/css2/webfonts/TripSans/TripSans-VF.woff2?v1.002" rel="preload" as="font" type="font/woff2" crossorigin="anonymous"/><link rel="icon" id="favicon" href="https://static.tacdn.com/favicon.ico?v2" type="image/x-icon"/><link rel="mask-icon" sizes="any" href="https://static.tacdn.com/img2/brand_refresh/application_icons/mask-icon.svg" color="#000000"/><meta name="theme-color" content="#34e0a1"/><meta name="viewport" content="width=device-width, initial-scale=1.0, viewport-fit=cover"/><meta name="format-detection" content="telephone=no"/><meta property="al:ios:app_name" content="TripAdvisor"/><meta property="al:ios:app_store_id" content="284876795"/><meta property="twitter:app:id:ipad" name="twitter:app:id:ipad" content="284876795"/><meta property="twitter:app:id:iphone" name="twitter:app:id:iphone" con

In [106]:
soup = bs(ride_review.content, 'html.parser')

In [107]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="en" http-equiv="content-language"/>
  <link as="font" crossorigin="anonymous" href="https://static.tacdn.com/css2/webfonts/TripSans/TripSans-VF.woff2?v1.002" rel="preload" type="font/woff2"/>
  <link href="https://static.tacdn.com/favicon.ico?v2" id="favicon" rel="icon" type="image/x-icon"/>
  <link color="#000000" href="https://static.tacdn.com/img2/brand_refresh/application_icons/mask-icon.svg" rel="mask-icon" sizes="any"/>
  <meta content="#34e0a1" name="theme-color"/>
  <meta content="width=device-width, initial-scale=1.0, viewport-fit=cover" name="viewport"/>
  <meta content="telephone=no" name="format-detection"/>
  <meta content="TripAdvisor" property="al:ios:app_name"/>
  <meta content="284876795" property="al:ios:app_store_id"/>
  <meta content="284876795" name="twitter:app:id:ipad" property="twitter:app:id:ipad"/>
  <meta content="284876795" name="t

In [ ]:
review_div = soup.findAll(name = "div", attrs = {"class": "dDwZb"})

In [ ]:
review_div

In [ ]:
review_div_2 = soup.findAll(name = "div", attrs = {"class": "cjhIj"})

In [ ]:
review_div_2

In [ ]:
review_div_3 = soup.findAll(name = "div", attrs = {"class": "fCFzN"})

In [ ]:
review_div_3

In [ ]:
review_div_4 = soup.findAll(name = "div", attrs = {"class": "duhwe"})

In [18]:
review_div_4

[<div class="duhwe _T bOlcm dMbup"><div class="pIRBV _T KRIav" style="-webkit-line-clamp:7;line-break:normal;cursor:pointer"><div class="WlYyy diXIH dDKKM"><span class="NejBf">So good! The constant changing of which planets you visit helps to make this a ride we visit several times during our trip!</span></div></div><div class="eIVRK"><button class="bfQwA _G B- _S _T c G_ P0 ddFHE cnvzr" type="button"><span class="WlYyy cacGK Wb">Read more</span><div class="bLMrm"><svg class="fecdL d Vb wQMPa" height="20px" viewbox="0 0 24 24" width="20px"><path d="M18.4 7.4L12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></button></div></div>,
 <div class="duhwe _T bOlcm dMbup"><div class="pIRBV _T KRIav" style="-webkit-line-clamp:7;line-break:normal;cursor:pointer"><div class="WlYyy diXIH dDKKM"><span class="NejBf">Star Tours is the oldest Star Wars ride at Disney Hollywood studios, but it’s still a very entertaining ride. The wait time for the ride was no longer than 20 minutes during m

In [ ]:
review_div_5 = soup.findAll(name = "svg", attrs = {"class": "RWYkj d H0"})

In [27]:
review_div_5

[<svg aria-label="" class="RWYkj d H0" height="16" title="" viewbox="0 0 88 16" width="88"><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(18 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(36 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(54 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(72 0) scale(0.6666666666666666)"></path></svg>,
 <svg aria-label="5.0 of 5 bubbles" class="RWYkj d H0" height="12" title="5.0 of 5 bubbles" viewbox="0 0 

In [24]:
len(review_div_5)

74

bs4.element.ResultSet

In [21]:
review_div_6

NameError: name 'review_div_6' is not defined

In [9]:
ride_review_div = soup.findAll(name = "div", attrs = {"aria-label=": " of 5 bubbles"})

In [10]:
ride_review_div

[]

### A for loop method that didn't end up working out.

In [ ]:
for review in soup.findAll('a',{'class':'review_count'}):
  a = review['href']
  a = 'https://www.tripadvisor.'+ a
  a = a[:(a.find('Reviews')+7)] + '-or{}' + a[(a.find('Reviews')+7):]
  print(a)
  links.append(a)

In [ ]:
for i in range(0,200,10):
     url = (f'https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or{0}-Star_Tours_The_Adventures_Continue-Orlando_Florida.html')
     page = requests.get(url)
     soup = BeautifulSoup(page.content, 'html.parser')
     results = soup.find('div', { 'id' : 'cmp-container'})
     elems = results.find_all(class_='cmp-Review-container')
     for elem in elems:
         title = elem.find(attrs = {'class':'cmp-Review-title'})
         review = elem.find('div', {'class': 'cmp-Review-text'})
         author = elem.find(attrs = {'class':'cmp-Review-author'})
         rating = elem.find(attrs = {'class':'cmp-ReviewRating-text'})
         df = df.append({'review_title': title.text,
          'review': review.text,
        'author': author.text,
          'rating': rating.text
            }, ignore_index=True)

### Trying to Figure Out Review Bubbles

In [ ]:
<svg class="RWYkj d H0" viewBox="0 0 88 16" width="88" height="16" aria-label="5.0 of 5 bubbles" title="5.0 of 5 bubbles"><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(18 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(36 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(54 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(72 0) scale(0.6666666666666666)"></path></svg>

In [ ]:
<div class="RTVWf o W f u w eeCyE" aria-label="4.5 of 5 bubbles. 306 reviews"><svg class="RWYkj d H0" viewBox="0 0 88 16" width="88" height="16" aria-label="" title=""><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(18 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(36 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(54 0) scale(0.6666666666666666)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(72 0) scale(0.6666666666666666)"></path></svg><span class="WlYyy diXIH bGusc dDKKM"><span class="cfIVb">306</span></span></div>

### Another Selenium attempt that didn't work.

In [35]:
!pip install selenium

In [39]:
import sys
import csv
from selenium import webdriver
import time

# default path to file to store data
path_to_file = "/Users/Christopher.Doughty/Downloads/Data Science/Notebooks/project eda brief/Scraping-TripAdvisor-with-Python-2020/ridereviews.csv"

# default number of scraped pages
num_page = 20

# default tripadvisor website of restaurant
url = "https://www.tripadvisor.com/Attraction_Review-g34515-d8563504-Reviews-or30-Star_Tours_The_Adventures_Continue-Orlando_Florida.html"

# if you pass the inputs in the command line
if (len(sys.argv) == 4):
    path_to_file = sys.argv[1]
    num_page = int(sys.argv[2])
    url = sys.argv[3]

# Import the webdriver
driver = webdriver.Firefox()
driver.get(url)

# Open the file to save the review
csvFile = open(path_to_file, 'a', encoding="utf-8")
csvWriter = csv.writer(csvFile)

# change the value inside the range to save more or less reviews
for i in range(0, num_page):
    
    # expand the review 
    time.sleep(2)
    driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']").click()

    container = driver.find_elements_by_xpath(".//div[@class='review-container']")

    for j in range(len(container)):

        title = container[j].find_element_by_xpath(".//span[@class='noQuotes']").text
        date = container[j].find_element_by_xpath(".//span[contains(@class, 'ratingDate')]").get_attribute("title")
        rating = container[j].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
        review = container[j].find_element_by_xpath(".//p[@class='partial_entry']").text.replace("\n", " ")

        csvWriter.writerow([date, rating, title, review]) 

    # change the page
    driver.find_element_by_xpath('.//a[@class="nav next ui_button primary"]').click()

driver.close()

WebDriverException: Message: 'geckodriver' executable needs to be in PATH. 
